In [1]:
using CairoMakie
using Unitful
using Colors
import PhysicalConstants.CODATA2018: h, k_B, c_0

In [16]:
# Set some plot defaults
custom_theme = Theme(
    Axis = (
        xgridvisible = false,  # Set grid visibility to false
        ygridvisible = false,
    ),
)
Makie.set_theme!(custom_theme)

In [17]:
i_units = u"kW / (m^2 * nm * sr)"

function blackbody_λ(λ, temp) 
    radiation = 2h * c_0^2 * λ^-5 / (exp(h * c_0 / k_B / (λ * temp)) - 1)
    return radiation |> i_units
end

if_units = u"nW / (m^2 * Hz * sr)"

function blackbody_ν(ν, temp) 
    radiation = 2h * ν^3 / c_0^2 / (exp(h * ν / (k_B * temp)) - 1)
    return radiation |> if_units
end

blackbody_ν (generic function with 1 method)